In [1]:
import pandas as pd 
import numpy as np 
import anndata as an 
import sys, os 
sys.path.append('../src')

import sys
import os
import pathlib 
from typing import *

import torch
import numpy as np 
import pandas as pd 
import anndata as an
from functools import cached_property

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.preprocessing import LabelEncoder 

from torchmetrics.functional import *

from data import *
from lightning_train import *
from model import *

In [11]:
# currgenes = an.read_h5ad('../data/dental/human_dental_T.h5ad').var.index
# refgenes = currgenes[5:]

module = DataModule(
    datafiles=['../data/dental/human_dental_T.h5ad'],
    labelfiles=['../data/dental/labels_human_dental.tsv'],
    class_label='cell_type',
    sep='\t',
#     refgenes=refgenes,
#     currgenes=currgenes,
    batch_size=16,
    num_workers=0,
    subset=list(range(500)),
)

module.setup()

model = TabNetLightning.load_from_checkpoint(
    '../checkpoints/checkpoint-80-desc-dental.ckpt',
    input_dim=module.num_features,
    output_dim=module.num_labels,
)

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights
Initializing network
Initializing explain matrix


In [14]:
from tqdm import tqdm

def confusion_matrix(model, dataloader, num_classes):
    confusion_matrix = torch.zeros(num_classes, num_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(tqdm(dataloader)):
            outputs, _ = model(inputs)
            
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    return confusion_matrix 
mat = confusion_matrix(model, module.trainloader, model.output_dim)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 10.36it/s]


In [16]:
mat

tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   1.],
        [  0.,   0.,  12.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   3.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   5.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  1.,   0.,   0.,   0.,   5.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   6.,   0.],
        [ 11.,   0.,  61.,  43.,  18.,   0.,   0.,  28., 125.]])

In [5]:
early_stop_callback = EarlyStopping(
    monitor="weighted_val_accuracy", 
    min_delta=0.00, 
    patience=3, 
    verbose=False, 
    mode="max"
)

model = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.001,
    },
    metrics={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
    weighted_metrics=True,
)

wandb_logger = WandbLogger(
    project=f"refgenes-test",
    name='Dental model'
)

trainer = pl.Trainer(
    logger=wandb_logger,
    callbacks=[early_stop_callback],
    max_epochs=200,
)

trainer.fit(model, datamodule=module)

Initializing network
Initializing explain matrix


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 1.4 M 
-----------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.769     Total estimated model params size (MB)


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
